In [7]:
#%%
import torch
from torch_geometric.datasets import Planetoid
from torch_geometric.data import DataLoader
from torch_geometric.nn.models import VGAE


In [16]:
import torch_geometric

In [17]:
torch_geometric.__version__

'2.6.1'

In [18]:
dataset = Planetoid(root="data/Cora", name="Cora")

In [21]:
import anomaly_injection

In [22]:
data = dataset[0]

In [29]:
data.edge_index[:,2]

tensor([2582,    0])

In [ ]:
new_data, struct_anomalies = anomaly_injection.structurally_perturb(data, 5, 30)
new_new_data, att_anomalies = anomaly_injection.attribute_perturb(new_data, 150)

Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resampling...
Resamp

In [44]:
torch.concat([struct_anomalies.flatten(), att_anomalies])

tensor([1518, 2323, 1679, 1954, 1111, 1010,  716,  664,  943, 1834,  665, 1266,
        1440, 1566, 2696, 1710,  617,   97,  826, 2448,  516, 2474, 1199, 1251,
          79, 1594,   69,  433, 1232,  339,  354, 1405, 1980, 1760, 1324,  779,
        1909, 2221, 2104, 1515,  792,  378, 2220, 1692, 1387,  279, 1101,  601,
         626, 1879, 1764, 1833,  595, 1605,  775,  866,   29,  519, 1416,   38,
         611,  557, 1968, 2306,  146, 1963, 2296,  363, 1769,  210, 1598,   85,
        1876, 1979, 1417,   84,  456, 2386,  281, 1576,  642,  603, 2638, 1647,
         313, 1411, 1986,  518,  105, 2195,  631, 2677, 1002, 2010, 1795, 2215,
        1613,  449, 2698, 2120, 2152, 2172, 2121, 2132, 1937, 1779,  732,  302,
        1976, 2608, 1905, 1874, 1941,  543, 1072,  883, 1060, 2496, 1340, 2671,
        1024, 2151, 1054, 1312, 1611, 1959,  613,  698, 1281, 1550, 1208, 2493,
        1457,  519,  395,  451, 1299, 1427,  343, 1259,  445,  879, 1844,  441,
        1559, 2147,  709, 1353,  523, 20

In [11]:

print(type(dataset))
# %%
data = dataset[0]

print(data.num_nodes)
print(data.keys())

<class 'torch_geometric.datasets.planetoid.Planetoid'>
2708
['train_mask', 'y', 'val_mask', 'edge_index', 'test_mask', 'x']


In [12]:
data.edge_index

tensor([[ 633, 1862, 2582,  ...,  598, 1473, 2706],
        [   0,    0,    0,  ..., 2707, 2707, 2707]])

In [3]:
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges

In [4]:
class VariationalGCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(VariationalGCNEncoder, self).__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels, cached=True) # cached only for transductive learning
        self.conv_mu = GCNConv(2 * out_channels, out_channels, cached=True)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels, cached=True)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv_mu(x, edge_index), self.conv_logstd(x, edge_index)

In [5]:
data.edge_index.shape

torch.Size([2, 114615892])

In [6]:
out_channels = 2
num_features = dataset.num_features
epochs = 300


model = VGAE(VariationalGCNEncoder(num_features, out_channels))  # new line

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
x = data.x.to(device)

train_edge_mask = data.train_mask[data.edge_index[0]] & data.train_mask[data.edge_index[1]]

train_pos_edge_index = data.edge_index[:,train_edge_mask].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [7]:
def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(x, train_pos_edge_index)
    loss = model.recon_loss(z, train_pos_edge_index)
    
    loss = loss + (1 / data.num_nodes) * model.kl_loss()  # new line
    loss.backward()
    optimizer.step()
    return float(loss)


def test(pos_edge_index, neg_edge_index):
    model.eval()
    with torch.no_grad():
        z = model.encode(x, train_pos_edge_index)
    return model.test(z, pos_edge_index, neg_edge_index)

In [8]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
writer = SummaryWriter('runs/GAE1_experiment_'+'2d_100_epochs')
for epoch in range(1, epochs + 1):
    loss = train()
    auc, ap = test(data.test_pos_edge_index, data.test_neg_edge_index)
    print('Epoch: {:03d}, AUC: {:.4f}, AP: {:.4f}'.format(epoch, auc, ap))
    
    
    writer.add_scalar('auc train',auc,epoch) # new line
    writer.add_scalar('ap train',ap,epoch)   # new line

In [ ]:

#%%



# data.edge_index is 2 by num_edges tensor 
# column = [i,j] means there is an edge from node i to node j
# data.y is the labels 
#   - not predicting this label. can add as feature or ignore
# data.x is n by d, where n is number of nodes and d is number of features

# how are we using train, val, test split?

# TODO: encode and decode data
# TODO: make anomaly detector for data (missing/new edges, significantly different features)
#       - sort all nodes, most to least likely to be anomaly?

# TODO: 


# might be helpful: https://github.com/Flawless1202/VGAE_pyG/
# also, the VGAE() class
# https://github.com/DaehanKim/vgae_pytorch
# https://antoniolonga.github.io/Pytorch_geometric_tutorials/posts/post6.html
#%%
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges


#%%

# GAN: https://github.com/hwwang55/GraphGAN
# https://arxiv.org/abs/1711.08267
# https://medium.com/@_psycoplankton/graphgan-generative-adversarial-networks-for-graphs-ff4584375a81



def rank_anomalous(original_data, reconstructed_data):
    
    # score for a node: something like
    # num new edges (or edge deletions) with that node + || original feature - reconstructed feature||
    # or something
    
    scores = [0]*original_data.num_nodes
    for i in range(original_data.num_nodes):
        scores += norm of original_data[i]-reconstructed_data[i]
    
    del_edges = (original_data.edge_index setminus reconstructed_data.edge_index)
    ins_edges = (reconstructed_data.edge_index setminus original_data.edge_index)
    for edge in del_edges.union(ins_edges):
        scores[edge[0]] += 1
        scores[edge[1]] += 1
    
    indices = list(range(original_data.num_nodes))
    indices.sort(key = lambda i: -scores[i])
    return indices